In [34]:
import pandas as pd
import numpy as np
import math

### Carga dos dados

In [35]:
# definir arquivo Pickle de entrada
#arquivo = 'dados/afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'dados/cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [36]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [37]:
df.shape

(41146, 18)

### Expansão dos dados agregados

In [38]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

41146 1 20389 143
CPU times: user 341 ms, sys: 0 ns, total: 341 ms
Wall time: 339 ms


,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0


In [39]:
# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
df = dfe # apontar para o dataframe expandido

### Uso dos dados originais

In [40]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (82709, 16) (82709,)


In [41]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

print("Shapes:\n"\
      "   Train      - X: %s, y: %s\n" \
      "   Test       - X: %s, y: %s" %
      (X_train.shape, y_train.shape, X_test.shape, y_test.shape))

Shapes:
   Train      - X: (57896, 16), y: (57896,)
   Test       - X: (24813, 16), y: (24813,)


In [42]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
ANDERLAN,14,1,12,18,5,4,14,1,0,0,0,0,0,0,0,0
AURIA,1,9,18,21,1,0,0,0,0,0,0,0,0,0,0,0
GILSON,14,15,19,12,9,7,0,0,0,0,0,0,0,0,0,0
THALITA,1,20,9,12,1,8,20,0,0,0,0,0,0,0,0,0
ERLEY,25,5,12,18,5,0,0,0,0,0,0,0,0,0,0,0


In [43]:
y_train.head()

NOME
ANDERLAN    1
AURIA       0
GILSON      1
THALITA     0
ERLEY       1
Name: SEXO, dtype: uint8

In [44]:
import os
os.environ['PYTHONHASHSEED']=str(1)

import tensorflow as tf
import random
import numpy as np

def reset_random_seeds(SEED = 42):
  os.environ['PYTHONHASHSEED']=str(SEED)
  tf.random.set_seed(SEED)
  np.random.seed(SEED)
  random.seed(SEED)

In [45]:
import tensorflow as tf
import autokeras as ak

In [46]:
# Initialize the structured data classifier
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=10 # It tries 3 different models
)
clf

In [49]:
X_train.to_numpy().astype('int8')

array([[14,  1, 12, ...,  0,  0,  0],
       [ 1,  9, 18, ...,  0,  0,  0],
       [14, 15, 19, ...,  0,  0,  0],
       ...,
       [ 9, 14,  1, ...,  0,  0,  0],
       [15,  3, 19, ...,  0,  0,  0],
       [ 1,  9, 12, ...,  0,  0,  0]], dtype=int8)

In [50]:
y_train.to_numpy().astype('int8')

array([1, 0, 1, ..., 0, 1, 0], dtype=int8)

In [51]:
%%time

# Feed the structured data classifier with training data
clf.fit(
    X_train.to_numpy().astype('int8'),
    y_train.to_numpy().astype('int8'),
    validation_split=0.15, # Split the training data and use the last 15% as validation data
    #epochs=10, # leave the epochs unspecified for an adaptive number of epochs
)

Trial 10 Complete [00h 02m 53s]
val_accuracy: 0.9913274645805359

Best val_accuracy So Far: 0.9924837946891785
Total elapsed time: 00h 24m 44s
INFO:tensorflow:Oracle triggered exit
Epoch 1/62
1810/1810 [==============================] - 2s 987us/step - loss: 0.1907 - accuracy: 0.9248
Epoch 2/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.1481 - accuracy: 0.9451
Epoch 3/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.1259 - accuracy: 0.9543
Epoch 4/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.1060 - accuracy: 0.9670
Epoch 5/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.0865 - accuracy: 0.9745
Epoch 6/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.0740 - accuracy: 0.9785
Epoch 7/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.0651 - accuracy: 0.9805
Epoch 8/62
1810/1810 [==============================] - 2s 1ms/step - loss: 0.0603 - accuracy: 0.9818
E

In [52]:
# evaluate the model
loss, acc = clf.evaluate(X_test, y_test, verbose=0)
print('Acurácia: %.2f %%' % (acc * 100))

Acurácia: 99.16 %


In [53]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 16)                0         
_________________________________________________________________
normalization (Normalization (None, 16)                33        
_________________________________________________________________
dense (Dense)                (None, 1024)              17408     
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                32800     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [54]:
type(clf)

autokeras.tasks.structured_data.StructuredDataClassifier

In [55]:
type(model)

tensorflow.python.keras.engine.functional.Functional

In [58]:
model = clf.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

'''try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")'''

<class 'tensorflow.python.keras.engine.functional.Functional'>


'try:\n    model.save("model_autokeras", save_format="tf")\nexcept Exception:\n    model.save("model_autokeras.h5")'

In [59]:
# save model to file
model.save('modelos/modelo-genero-nome-autokeras', save_format="tf")
#model.save('modelos/modelo-genero-nome-autokeras.h5')

INFO:tensorflow:Assets written to: modelos/modelo-genero-nome-autokeras/assets


In [60]:
# carregar um modelo previamente gravado em disco
from tensorflow.keras.models import load_model

model = load_model('modelos/modelo-genero-nome-autokeras', custom_objects=ak.CUSTOM_OBJECTS)
#model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
#model = load_model('modelos/modelo-genero-nome-tensorflow.h5')
model

In [61]:
df2 = X_test.copy()
df2['PROB'] = model.predict(X_test)
df2['REAL'] = y_test
df2['PREV'] = df2['PROB'].apply(lambda x: 1 if x > 0.5 else 0)
df2['DIF'] = df2[['REAL', 'PREV']].apply(lambda x: abs(x[0] - x[1]), axis=1)
#df2['REAL'].apply(lambda x: 'F' if x else 'M')
#df2['PREV'].apply(lambda x: 'F' if x else 'M')
df2.head(15)

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,PROB,REAL,PREV,DIF
NOME,,,,,,,,,,,,,,,,,,,,
LICIO,15,9,3,9,12,0,0,0,0,0,0,0,0,0,0,0,9.999110e-01,1,1,0
SOLANGE,5,7,14,1,12,15,19,0,0,0,0,0,0,0,0,0,1.984843e-06,0,0,0
BERTHA,1,8,20,18,5,2,0,0,0,0,0,0,0,0,0,0,2.932475e-09,0,0,0
VALDENILDA,1,4,12,9,14,5,4,12,1,22,0,0,0,0,0,0,1.331653e-34,0,0,0
JUVANETE,5,20,5,14,1,22,21,10,0,0,0,0,0,0,0,0,8.840263e-13,0,0,0
THAISA,1,19,9,1,8,20,0,0,0,0,0,0,0,0,0,0,2.577006e-08,0,0,0
OLINTO,15,20,14,9,12,15,0,0,0,0,0,0,0,0,0,0,9.999515e-01,1,1,0
TRICIA,1,9,3,9,18,20,0,0,0,0,0,0,0,0,0,0,1.348125e-21,0,0,0
SANANE,5,14,1,14,1,19,0,0,0,0,0,0,0,0,0,0,7.128709e-08,0,0,0


In [62]:
df2[df2['DIF'] > 0].head(15)

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,PROB,REAL,PREV,DIF
NOME,,,,,,,,,,,,,,,,,,,,
WYLLIANS,19,14,1,9,12,12,25,23,0,0,0,0,0,0,0,0,3.483057e-03,1,0,1
WIHARLLEY,25,5,12,12,18,1,8,9,23,0,0,0,0,0,0,0,2.605915e-02,1,0,1
JASPE,5,16,19,1,10,0,0,0,0,0,0,0,0,0,0,0,1.147033e-01,1,0,1
GLAUCIELLHO,15,8,12,12,5,9,3,21,1,12,7,0,0,0,0,0,2.622497e-02,1,0,1
IASMIM,13,9,13,19,1,9,0,0,0,0,0,0,0,0,0,0,1.000000e+00,0,1,1
EVLYN,14,25,12,22,5,0,0,0,0,0,0,0,0,0,0,0,7.888044e-01,0,1,1
TOMIL,12,9,13,15,20,0,0,0,0,0,0,0,0,0,0,0,3.980015e-01,1,0,1
SILUE,5,21,12,9,19,0,0,0,0,0,0,0,0,0,0,0,3.310199e-07,1,0,1
THAMILES,19,5,12,9,13,1,8,20,0,0,0,0,0,0,0,0,4.853693e-02,1,0,1
